<a href="https://colab.research.google.com/github/jovanape/Bitcoint-Price-Prediction/blob/main/rnn_close.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predvidjanje vrednosti Bitcoina pomocu RNN**

**Atributi:** Close

**Filtrirani podaci:** od 28.03.2021.

## **Ucitavanje biblioteka**

In [14]:
# Ucitavanje biblioteka

import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import callbacks

## **Ucitavanje skupa podataka i pretprocesiranje**

In [15]:
# Ucitavanje skupa podataka

data = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
data.head()

ParserError: ignored

In [ ]:
# Ciscenje podataka od NaN vrednosti

data = data.dropna()
data.head()

In [ ]:
# Kreiranje datuma

data['Date'] = pd.to_datetime(data['Timestamp'], unit='s').dt.date # samo datum (bez vremena)


# Filtriranje podataka

data = data.loc[data['Date'] > pd.to_datetime('2021-03-28')]
data.head(4)

In [ ]:
# Dimenzije DataFrame-a

data.shape

## **Podela skupa, vizualizacija**



### **Podela skupa podataka**

In [ ]:
# Definicija funkcije za podelu podataka na train i test skup

def data_split(data, test_size = 0, days_to_predict = 0):

  if test_size == 0 and days_to_predict == 0:
    print('Uneti vrednost za jedan od parametara test_size ili days_to_predict.\n')
    return
  elif test_size == 0: # dato je days_to_predict
    test_limit = len(data) - prediction_days
  else: # dato je test_size ili oba parametra
    test_limit = len(data) - int(test_size * len(data))

  train = data[:test_limit]
  test = data[test_limit:]
  
  return train, test

In [ ]:
# Podela inicijalnog skupa podataka na train i test skup

train, test = data_split(data, test_size = 0.2, days_to_predict = 0)

In [ ]:
# Dimenzije train i test skupa

train.shape, test.shape

### **Vizualizacija podataka**

In [ ]:
# Definicija funkcije za vizuelni prikaz train i test podataka

def plot_data(train_data = train['Close'], test_data = test['Close']):

  fig, ax = plt.subplots(1, figsize=(16, 9))
  ax.plot(train_data, label='trening', linewidth=2)
  ax.plot(test_data, label='test', linewidth=2)
  ax.set_ylabel('cena [USD]', fontsize=14)
  ax.set_title(label = 'Vizuelni prikaz skupova za trening i test', fontsize=18)
  ax.legend(loc='best', fontsize=18)

  #data = data.reset_index()
  #x = data.index
  #ax.set_xticks(range(87724))


In [ ]:
# Plotovanje train i test skupa podataka

plot_data(train_data = train['Close'], test_data = test['Close'])

## **Priprema trening skupa**

## **Filtriranje atributa train skupa**

In [ ]:
# Izbacivanje nepotrebnih kolona (atributa) iz train skupa

train_ = train.drop(['Low', 'High', 'Open', 'Date', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price', 'Timestamp'], axis = 1)
train_.head()

In [ ]:
# Dimenzije train skupa - pre i nakon izbacivanja atributa

train.shape, train_.shape

## **Skaliranje train skupa**

In [ ]:
# Skaliranje trening skupa

scaler = MinMaxScaler(feature_range=(0, 1))
train_ = scaler.fit_transform(train_)
train_

## **Odredjivanje ciljne promenljive na train skupu**

In [ ]:
# Odredjivanje ciljne promenljive na train skupu

X_train = []
y_train = []

for i in range(60, train_.shape[0]):
    X_train.append(train_[i-60:i])
    y_train.append(train_[i,0])


# train.shape = 100
# i in (60, 100)
# x_train.append(train[i-60:i])
# y_train.append(train[i, 0])

# i = 61
# x_train - train[1:61]
# y_train - train[61, 0]

# i = 62
# x_train - train[1:61] + train[2:62]
# y_train - train[61, 0] + train[62, 0]

# ...

# i = 99
# x_train - train[1:61] + train[2:62] + ... + train[33:99]
# y_train - train[61, 0] + train[62, 0] + ... + train[99, 0]


In [ ]:
# Konverzija u np.array

X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Dimenzije X_train

X_train.shape

## **Kreiranje, kompilacija i treniranje mreze**

### **Kreiranje mreze**

In [ ]:
# Definicija funkcije koja kreira i vraca model mreze

def build_net(input_1 = X_train.shape[1], input_2 = X_train.shape[2]):

  net = Sequential()
  net.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (input_1, input_2)))
  net.add(Dropout(0.2))
  net.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
  net.add(Dropout(0.3))

  net.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
  net.add(Dropout(0.4))

  net.add(LSTM(units = 120, activation = 'relu'))
  net.add(Dropout(0.5))

  net.add(Dense(units =1))
  print(net.summary())

  return net


In [ ]:
# Kreiranje modela mreze

net = build_net(input_1 = X_train.shape[1], input_2 = X_train.shape[2])

### **Kompilacija i treniranje mreze**

In [ ]:
# Definicija funkcije za kompilaciju i treniranje mreze
# Treniranje je moguce vrsiti sa ili bez validacije

def compile_and_fit(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'accuracy', X_train_ = X_train, y_train_ = y_train, epochs = 20, validation = 0.1, batch_size = 50, patience=3):

  net.compile(optimizer = optimizer, loss = loss, metrics = metrics)

  callback = callbacks.EarlyStopping(monitor = 'loss', patience = patience)

  if validation == 0:
    history = net.fit(X_train_, y_train_, epochs = epochs, batch_size = batch_size, callbacks=[callback])
  else:
    history = net.fit(X_train_, y_train_, epochs = epochs, validation_split = validation, batch_size = batch_size, callbacks=[callback])

  return history, net


In [ ]:
# Kompilacija i treniranje mreze sa validacijom

history_validation, net_validation = compile_and_fit(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy', 'mae'], X_train_ = X_train, y_train_ = y_train, epochs = 20, validation = 0.1, batch_size = 50, patience=3)


# Kompilacija i treniranje mreze bez validacije

history_no_validation, net_no_validation = compile_and_fit(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy', 'mae'], X_train_ = X_train, y_train_ = y_train, epochs = 20, validation = 0, batch_size = 50, patience=3)


## **Priprema test skupa**

### **Filtriranje atributa test skupa**

In [ ]:
# Izbacivanje nepotrebnih kolona (atributa) iz test skupa

test_ = test.drop(['Low', 'High', 'Open', 'Date', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price', 'Timestamp'], axis = 1)


### **Skaliranje test skupa**

In [ ]:
# Skaliranje test skupa

test_ = scaler.transform(test_)

### **Odredjivanje ciljne promenljive na test skupu**

In [ ]:
# Odredjivanje ciljne (target) promenljive na test skupu

X_test = []
y_test = []

for i in range(60, test_.shape[0]):
    X_test.append(test_[i-60:i])
    y_test.append(test_[i,0])
    

In [ ]:
# Konverzija u np.array

X_test, y_test = np.array(X_test), np.array(y_test)

## **Predvidjanje**

In [ ]:
# Predvidjanje za mrezu treniranu bez i sa validacijom

# Bez validacije
y_predicted_no_validation = net_no_validation.predict(X_test)
print('Predvidjene vrednosti (prvih 10) - bez validacije:\n', y_predicted_no_validation[:10])

# Sa validacijom
y_predicted_validation = net_validation.predict(X_test)
print('\nPredvidjene vrednosti (prvih 10) - sa validacijom:\n', y_predicted_validation[:10])

## **Vizualizacija rezultata predvidjanja i metrika**

### **Vizuelni prikaz predvidjene i stvarne vrednosti**

In [ ]:
# Vizuelni prikaz rezultata modela mreze

def plot_real_vs_predicted_data(test_set, predicted_set):

  fig, ax = plt.subplots(1, figsize=(16, 9))
  ax.plot(test_set, label='Stvarna vrednost BTC', linewidth=2)
  ax.plot(predicted_set, label='Predvidjena vrednost BTC', linewidth=2)
  ax.set_xlabel('Redni broj instance', fontsize=14)
  ax.set_ylabel('Cena [USD]', fontsize=14)
  ax.set_title(label = 'Vizuelni prikaz stvarne i predvidjene vrednosti BTC', fontsize=18)
  ax.legend(loc='best', fontsize=18)

  # test = test.reset_index()
  # x = test.index


In [ ]:
# Plotovanje stvarne i predvidjene vrednosti

# Bez validacije
plot_real_vs_predicted_data(test_set = y_test, predicted_set = y_predicted_no_validation)

# Sa validacijom
plot_real_vs_predicted_data(test_set = y_test, predicted_set = y_predicted_validation)

### **Vizuelni prikaz gubitka, tacnosti i srednje apsolutne greske tokom treniranja mreze (sa i bez validacije)**

In [ ]:
# Vizuelni prikaz gubitka

def plot_loss(history_):

  epochs = range(len(history_.history['loss'])) # broj epoha
  plt.title('Vizuelni prikaz gubitka (loss)')
  plt.xlabel('broj epohe')
  plt.ylabel('gubitak')

  plt.plot(epochs, history_.history['loss'], label='Gubitak (loss) tokom treniranja')

  if 'val_loss' in history_.history:
    plt.plot(epochs, history_.history['val_loss'], label='Gubitak (val_loss) tokom treniranja sa validacijom')

  plt.legend(loc = 'best')

  plt.show()


In [ ]:
# Vizuelni prikaz tacnosti

def plot_accuracy(history_):

  epochs = range(len(history_.history['accuracy'])) # broj epoha
  plt.title('Vizuelni prikaz tacnosti (accuracy')
  plt.xlabel('broj epohe')
  plt.ylabel('tacnost')
  
  plt.plot(epochs, history_.history['accuracy'], label='Tacnost (accuracy) tokom treniranja')

  if 'val_accuracy' in history_.history:
    plt.plot(epochs, history_.history['val_accuracy'], label='Tacnost (val_accuracy) tokom treniranja sa validacijom')

  plt.legend(loc = 'best')

  plt.show()


In [ ]:
# Vizuelni prikaz MAE

def plot_mae(history_):

  epochs = range(len(history_.history['mae'])) # broj epoha
  plt.title('Vizuelni prikaz MAE (mae')
  plt.xlabel('broj epohe')
  plt.ylabel('MAE')
  
  plt.plot(epochs, history_.history['mae'], label='MAE (mae) tokom treniranja')

  if 'val_mae' in history_.history:
    plt.plot(epochs, history_.history['val_mae'], label='MAE (val_mae) tokom treniranja sa validacijom')

  plt.legend(loc = 'best')

  plt.show()


In [ ]:
# MAE

plot_mae(history_no_validation)

In [ ]:
# MAE - sa validacijom

plot_mae(history_validation)

In [ ]:
# Loss

plot_loss(history_no_validation)

In [ ]:
# Loss - sa validacijom

plot_loss(history_validation)

In [ ]:
# Accuracy

plot_accuracy(history_no_validation)

In [ ]:
# Accuracy - validation

plot_accuracy(history_validation)

In [ ]:
# Loss, accuracy, MAE nad skupom za testiranje

# (za model koji pri treniranju koristi validaciju
# i za model koji pri treniranju ne koristi validaciju)

test_metrics_no_val = net_no_validation.evaluate(X_test, y_test)
test_metrics_val = net_validation.evaluate(X_test, y_test)

print('\nMetrike prilikom testiranja - model treniran bez validacije\n')
print(net_no_validation.metrics_names[0], test_metrics_no_val[0])
print(net_no_validation.metrics_names[1], test_metrics_no_val[1])
print(net_no_validation.metrics_names[2], test_metrics_no_val[2])

print('\n\nMetrike prilikom testiranja - model treniran sa validacijom\n')
print(net_validation.metrics_names[0], test_metrics_val[0])
print(net_validation.metrics_names[1], test_metrics_val[1])
print(net_validation.metrics_names[2], test_metrics_val[2])


In [ ]:
# R2 skor nad skupom za testiranje

r2 = metrics.r2_score(y_predicted_no_validation, y_test)
r2_val = metrics.r2_score(y_predicted_validation, y_test)

print('R2 skor prilikom testiranja - model treniran bez validacije', r2)
print('R2 skor prilikom testiranja - model treniran sa validacijom', r2_val)